In [13]:
import pyDOE
from scipy import stats
import pandas as pd
import numpy as np
from scipy.stats import mannwhitneyu
import sys
from sklearn.preprocessing import MinMaxScaler
import scipy.stats.distributions as dist
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
from sklearn.gaussian_process.kernels import DotProduct
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from collections import namedtuple
from sklearn.linear_model import ElasticNet
import sklearn
from sklearn.preprocessing import PolynomialFeatures

## Surrogate Models

In [14]:
''' Elastic Net Regression '''
def elastic_net(train_data,test_data):
    scaler =  MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    regr = ElasticNet(alpha= 791.39 ,random_state=0 , l1_ratio=1.0, fit_intercept =True, max_iter=3000,selection='random').fit(scaler.transform ( np.array(train_data.iloc[:,:-1])) ,  np.array(train_data.iloc[:,-1]))
    pred = regr.predict(scaler.transform(test_data))
    return regr,pred


''' Kriging'''
def kriging(train_data,test_data):
    kernel =  RBF()
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    gpr = GaussianProcessRegressor(kernel=kernel,n_restarts_optimizer= 15,random_state=0,
                                   normalize_y=True ).fit(scaler.transform(train_data.iloc[:,:-1]), train_data.iloc[:,-1])
    pred = gpr.predict(scaler.transform(test_data))
    return gpr,pred

''' KNN Regression Implementation'''
def KNN(train_data,test_data):
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    regr = KNeighborsRegressor(n_neighbors=9,weights='distance',algorithm='brute',p=2
                               ).fit(scaler.transform(train_data.iloc[:,:-1]), train_data.iloc[:,-1])

    pred = regr.predict(scaler.transform(test_data))
    return regr,pred


''' Support Vector Regression'''
def _SVR(train_data,test_data):
    scaler = sklearn.preprocessing.MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    gpr = sklearn.svm.SVR(kernel='rbf', gamma = 4.96 , C = 1000 ,max_iter=1500).fit( scaler.transform(train_data.iloc[:,:-1]), train_data.iloc[:,-1])
    pred = gpr.predict(scaler.transform(test_data))
    return gpr,pred

""" Generating Polynomial Features i.e., Function Basis """
def quadratic_polynomial (df):
    return pd.DataFrame(PolynomialFeatures(degree=2).fit_transform(df))

""" Quadratic Regression with Elastic Net Penalty"""
def polynomial(tr, te):
    f_original = train['Y']
    true = test['Y']
    temp1 = quadratic_polynomial (tr.iloc[:,:-1])
    temp2 = quadratic_polynomial (te.iloc[:,:-1])
    temp1 ['Y'] = f_original
    model_eln , pred_eln = elastic_net(temp1,temp2)
    return model_eln , pred_eln

## Load Training and Test Data Set initially Generated

In [15]:
path = "Data Sets\\Original\\50 D\\Training_Data_Sets\\train_10_1000Samples.csv"
train_y = pd.read_csv(path).iloc[:,-1]
test_y = pd.read_csv(path[:-43]+str('Test_Data_Sets\\test_10_200Samples.csv')).iloc[:,-1]
path = "Data Sets\\Latent\\50 %\\50 D\\Training_Data_Sets\\latent_50D.csv"
train = pd.read_csv(path, index_col = 0)
test = pd.read_csv(path[:-33]+str('Test_Data_Sets\\latent_50D.csv'), index_col = 0)
train ['Y'] = train_y
test ['Y'] = test_y
del train_y
del test_y
true = np.array(test['Y'])

## Modeling Accuracy

## KNN

In [16]:
%timeit model_knn ,pred_knn = KNN(train,test.iloc[:,:-1])

7.85 ms ± 63.9 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [17]:
model_knn ,pred_knn = KNN(train,test.iloc[:,:-1])
np.mean((abs(true-pred_knn) / abs(true) ) * 100)

38.553069211821764

## SVR

In [18]:
%timeit model_svr , pred_svr = _SVR(train,test.iloc[:,:-1])

61.5 ms ± 226 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [19]:
model_svr , pred_svr = _SVR(train,test.iloc[:,:-1])
np.mean((abs(true-pred_svr) / abs(true) ) * 100)

40.38080080026674

## Kriging

In [20]:
%timeit model_kri , pred_kri = kriging(train,test.iloc[:,:-1])

6.52 s ± 51.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [21]:
model_kri , pred_kri = kriging(train,test.iloc[:,:-1])
np.mean((abs(true-pred_kri) / abs(true) ) * 100)

40.71361628214359

## ELN

In [22]:
%timeit polynomial (train,test)

C:\Users\sefi\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 625690654204064.0, tolerance: 38292438908020.56
  positive)
C:\Users\sefi\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 625690654204064.0, tolerance: 38292438908020.56
  positive)
C:\Users\sefi\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 625690654204064.0, tolerance: 38292438908020.56
  positive)
C:\Users\sefi\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:476: ConvergenceWarning

705 ms ± 34 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


C:\Users\sefi\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 625690654204064.0, tolerance: 38292438908020.56
  positive)


In [23]:
model_eln , pred_eln = polynomial (train,test)
np.mean((abs(true-pred_eln) / abs(true) ) * 100)

C:\Users\sefi\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 625690654204064.0, tolerance: 38292438908020.56
  positive)


40.7332688673667

## Save the Results

In [24]:
F2_accuracy = pd.DataFrame([true, pred_eln , pred_knn , pred_kri , pred_svr]).T
F2_accuracy.columns = ['True' , 'ELN' ,  'KNN' ,  'Kri',  'SVR']
F2_accuracy.to_csv('Results\\F10_Accuracy.csv')